In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
from tqdm import tqdm
import pickle
# %matplotlib inline

import keras
from keras.models import Model
from keras.layers import Dropout, Flatten, Conv2D, Input
from keras.datasets import mnist
from keras.utils import np_utils

from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler 

sys.path.append("/homes/lexjohan/Documents/EME/SVDD/")
from dataloader import unpack, unpack_ordered, unpack_polina
from modeldefault import VariationalAutoencoderModel

sys.path.append('/homes/lexjohan/Documents/models')
os.path.join("nxsdk_modules_ncl/snntoolbox")
from nxsdk_modules_ncl.dnn.src.utils import extract, to_integer
from nxsdk_modules_ncl.dnn.src.dnn_layers import NxInputLayer, NxDense, \
    NxModel, ProbableStates
from nxsdk_modules_ncl.dnn.composable.composable_dnn import ComposableDNN as DNN
from nxsdk_modules_ncl.input_generator.input_generator import InputGenerator
import nxsdk
from nxsdk.composable.model import Model
from nxsdk.logutils.nxlogging import set_verbosity,LoggingLevel
from nxsdk.api.enums.api_enums import ProbeParameter
from nxsdk.graph.monitor.probes import PerformanceProbeCondition

# Enable SLURM to run network on Loihi.
os.environ['SLURM'] = '1'
os.environ['PYTHONUNBUFFERED'] = '1'

os.environ["PARTITION"] = "nahuku32_2h"
os.environ["BOARD"] = 'ncl-ext-ghrd-01'

mode = 'ordered' #mode = 'polina' / mode = 'None'
Flags = None

data_path = "../EME/SVDD/data/"

train_path = data_path + "training.h5"
test_path = data_path + "testing.h5"

if mode == 'ordered':
    _, regression = unpack_ordered(train_path, Flags)
elif mode == 'polina':
    _, num_objects, regression = unpack_polina(train_path, Flags)
else:
    #load training and validation data
    _, num_objects, regression, classification = unpack(train_path, Flags)

scaler = StandardScaler()

scaler.fit(regression)
reg_normalized = scaler.transform(regression)

if mode == 'ordered':
    training_data = reg_normalized[:]
elif mode == 'polina':
    training_data = [num_objects[:], reg_normalized[:]]
else:
    training_data = [classification[:], reg_normalized[:]]

dataset_len = regression.shape[0]
if mode == 'ordered' or mode == 'polina':
    data_dim = regression.shape[1]
else:
    data_dim = classification.shape[1]

models_path = "../EME/SVDD/optimized_W/"

models = os.listdir(models_path)
model_name = models[0]
dim_z = int(model_name.split("zdim")[1].split(".")[0].split("_")[1])
ft = 0

hidden_layers = [512, 256, 128]

ann_model = VariationalAutoencoderModel(hidden_layers, model_name, data_dim, dataset_len, dim_z, ft, mode=mode, verbose=True)
ann_model.load_weights(models_path + model_name)

def run_model(num_steps_per_sample, train_data):
    batch_size = 64
    num_samples = train_data.shape[0]
    print("num_samples: {}".format(num_samples))
    print(f"time steps: {num_steps_per_sample * num_samples}")

    print("starting model run")
    tStart = time.time()
    snn_model.run(num_steps_per_sample * num_samples, aSync=True)
    tEndBoot = time.time()

    print("queueing data in input generator")
    with tqdm(total=num_samples) as pbar:
        for idx in range(0, num_samples, batch_size):
            batch = train_data[idx:idx+batch_size]
            b = np.abs(np.rint(batch).astype(int))
            input_generator.batchEncode(b)
            pbar.update(batch_size)

    tEndInput = time.time()
    print("Reading out channels")

    out = list(dnn.readout_channel.read(num_samples))
    tEndClassification = time.time()

    print("Finishing model run")
    snn_model.finishRun()
    
    return tStart, tEndBoot, tEndInput, tEndClassification, out



2023-01-11 11:34:58.831893: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 11:34:59.083281: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /slurm/intel-archi/lib
2023-01-11 11:34:59.083303: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-11 11:35:00.807298: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared obj

Unpacking file ../EME/SVDD/data/training.h5


event type int8 (3600000,) 3.6 MB
regression float32 (3600000, 76) 1094.4 MB


Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 in_regression (InputLayer)  [(None, 76)]              0         
                                                                 
 dense (Dense)               (None, 512)               39424     
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 z_mean (Dense)              (None, 21)                2709      
                                                                 
Total params: 206,357
Trainable params: 206,357
Non-trainable params: 0
_________________________________________________________________


2023-01-11 11:35:28.602463: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /slurm/intel-archi/lib
2023-01-11 11:35:28.604685: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-11 11:35:28.604700: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ncl-edu): /proc/driver/nvidia/version does not exist
2023-01-11 11:35:28.604918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
num_steps = 2

vth_mant = 2**9
bias_exp = 6
weight_exponent = 0
synapse_encoding = 'sparse'

in_regression = NxInputLayer(data_dim,
                            vThMant=vth_mant,
                            biasExp=bias_exp)

layer = NxDense(hidden_layers[0])(in_regression.input)
for idx in range(1, len(hidden_layers)):
    layer = NxDense(hidden_layers[idx])(layer)

layer = NxDense(dim_z)(layer)

set_verbosity(LoggingLevel.ERROR)
# Extract weights and biases from parameter list.
parameters = ann_model.model.get_weights()
weights = parameters[0::2]
biases = parameters[1::2]

# Quantize weights and biases using max-normalization (Strong quantization loss if distributions have large tails)
parameters_int = []
for w, b in zip(weights, biases):
    w_int, b_int = to_integer(w, b, 8)
    parameters_int += [w_int, b_int]

num_steps_per_img = 25
data_dim_snn = (data_dim, )

data_split = 0.01
num_test_samples = int(training_data.shape[0] * data_split)
execution_time_probe_bin_size = 512 # Small numbers slows down execution

for n in range(num_steps):

    snn_nxmodel = NxModel(in_regression.input, layer)
    # Set quantized weigths and biases for spiking model
    snn_nxmodel.set_weights(parameters_int)

    dnn = DNN(model=snn_nxmodel, num_steps_per_img=num_steps_per_img)

    input_generator = InputGenerator(shape=data_dim_snn, interval=num_steps_per_img)
    input_generator.setBiasExp(bias_exp)

    snn_model = nxsdk.composable.model.Model("dnn_model")
    snn_model.add(dnn)
    snn_model.add(input_generator)

    input_generator.connect(dnn)

    input_generator.processes.inputEncoder.executeAfter(dnn.processes.reset)
    snn_model.compile()

    eProbe = snn_model.board.probe(
        probeType=ProbeParameter.ENERGY, 
        probeCondition=PerformanceProbeCondition(
            tStart=1, 
            tEnd=num_test_samples*num_steps_per_img, 
            bufferSize=1024, 
            binSize=execution_time_probe_bin_size))

    snn_model.start(snn_nxmodel.board)
    tStart, tEndBoot, tEndInput, tEndClassification, out = run_model(num_steps_per_img, training_data[:num_test_samples])
    snn_model.disconnect()






x

x

..x

x

x

num_samples: 36000s... sters... 
time steps: 900000
starting model run
queueing data in input generator


36032it [00:44, 806.06it/s]                           


Reading out channels
Finishing model run
x   Processing timeseries... 

x

..x

x

x

num_samples: 36000s... sters... 
time steps: 900000
starting model run
queueing data in input generator


36032it [00:44, 806.37it/s]                           


Reading out channels
Finishing model run
